In [ ]:
# Configuration
from pathlib import Path
from utils.database.db_utils import get_db_connection
 
# Define the path to the DuckDB database file
project_root = Path.cwd().parent if "notebooks" in str(Path.cwd()) else Path.cwd()
db_path = project_root / "data" / "processed" / "chess_games.db"
 
# Define partitions for player_opening_stats
partitions = list("ABCDE") + ["other"]
 
# Print configuration details
print(f"Database path: {db_path}")
print(f"Partitions: {partitions}")

# Purpose

To decrease the size of our opening stats DB, making queries faster and more efficient.

# Methods:

    1. Change INTs to SMALLINT
        - player_id and opening_id
        - num_draws
            - Not doing this on num_wins and num_losses because those may exceed the limit of this data type (~32,000)
        - Not doing this with player_opening_stats ID because that's a composite string, not an int 
        - Saves 2 bytes per entry
    
    2. Change `color` VARCHAR to w/b enum
        - 1b
        - Compared to the 12-16bytes the previous varchar took up

# Considerations
    - Saving a copy of the DB in case I bork this

In [ ]:
# Print current database size
import os
try:
    db_size_bytes = os.path.getsize(db_path)
    db_size_mb = db_size_bytes / (1024 * 1024)
    print(f"Current database file size: {db_size_mb:.2f} MB")
except FileNotFoundError:
    print("Database file not found.")
    db_size_mb = 0

In [ ]:
# Execute migration pipeline
if db_size_mb > 0:
    with get_db_connection(db_path) as con:
        # First check max opening_id to ensure SMALLINT is safe
        max_opening_result = con.execute("SELECT MAX(id) FROM opening").fetchone()
        max_opening_id = max_opening_result[0] if max_opening_result else 0
        print(f"Max opening_id: {max_opening_id}")
        
        if max_opening_id > 32767:
            print("WARNING: opening_id exceeds SMALLINT range. Using INTEGER.")
            opening_id_type = "INTEGER"
        else:
            print("Safe to use SMALLINT for opening_id")
            opening_id_type = "SMALLINT"
        
        print("Creating new schema for partitioned tables...")
        con.execute("CREATE TYPE IF NOT EXISTS color_enum AS ENUM ('w', 'b');")

        # Keep player table as-is (INTEGER IDs)
        print("Creating new opening table with optimized ID type...")
        con.execute(f"""
            CREATE TABLE opening_new (
                id {opening_id_type} PRIMARY KEY,
                eco VARCHAR,
                name VARCHAR
            );
        """)

        # Migrate data to new opening table
        print("Migrating data to new opening table...")
        con.execute("INSERT INTO opening_new SELECT id, eco, name FROM opening;")

        # Create new player_opening_stats tables with ACTUALLY optimized schema
        print("Creating new player_opening_stats tables with optimized data types...")
        for letter in partitions:
            new_table = f"player_opening_stats_{letter}_new"
            con.execute(f"""
                CREATE TABLE {new_table} (
                    player_id   INTEGER NOT NULL,
                    opening_id  {opening_id_type} NOT NULL,
                    color       color_enum NOT NULL,
                    num_wins    INTEGER DEFAULT 0,
                    num_draws   SMALLINT DEFAULT 0,
                    num_losses  INTEGER DEFAULT 0,
                    PRIMARY KEY (player_id, opening_id, color)
                );
            """)

        # Migrate data from old player_opening_stats tables with proper casting
        print("Migrating player_opening_stats data with optimized types...")
        for letter in partitions:
            old_table = f"player_opening_stats_{letter}"
            new_table = f"player_opening_stats_{letter}_new"
            print(f"Migrating data from {old_table} to {new_table}...")
            con.execute(f"""
                INSERT INTO {new_table} (player_id, opening_id, color, num_wins, num_draws, num_losses)
                SELECT 
                    player_id,
                    CAST(opening_id AS {opening_id_type}),
                    CAST(CASE 
                        WHEN color = 'white' THEN 'w'::color_enum 
                        WHEN color = 'black' THEN 'b'::color_enum
                        WHEN color = 'w' THEN 'w'::color_enum
                        ELSE 'b'::color_enum 
                    END) as color,
                    num_wins,
                    CAST(num_draws AS SMALLINT),
                    num_losses
                FROM {old_table}
                WHERE num_wins <> 0 OR num_draws <> 0 OR num_losses <> 0;
            """)

        # Now drop old tables and rename new ones
        print("Swapping old tables with new tables...")
        for letter in partitions:
            old_table = f"player_opening_stats_{letter}"
            new_table = f"player_opening_stats_{letter}_new"
            con.execute(f"DROP TABLE {old_table};")
            con.execute(f"ALTER TABLE {new_table} RENAME TO {old_table};")

        # Drop old opening table and rename new one
        con.execute("DROP TABLE opening;")
        con.execute("ALTER TABLE opening_new RENAME TO opening;")

        print("Migration complete. Data optimized with proper type casting.")

In [ ]:
# Print new database size and vacuum
try:
    db_size_bytes = os.path.getsize(db_path)
    db_size_mb = db_size_bytes / (1024 * 1024)
    print(f"New database file size: {db_size_mb:.2f} MB")
 
    with get_db_connection(db_path) as con:
        print("Vacuuming database to optimize storage...")
        con.execute("VACUUM;")
        print("Vacuum complete.")
 
        db_size_bytes = os.path.getsize(db_path)
        db_size_mb = db_size_bytes / (1024 * 1024)
        print(f"Post-vacuum database file size: {db_size_mb:.2f} MB")
except FileNotFoundError:
    print("Database file not found.")

In [ ]:
# Force DuckDB to rewrite the file to reclaim space
with get_db_connection(db_path) as con:
    print("Forcing database file rewrite to reclaim space...")
    con.execute("CHECKPOINT;")
    con.execute("PRAGMA force_checkpoint;")
    print("Checkpoint complete.")

# Final size check
try:
    db_size_bytes = os.path.getsize(db_path)
    db_size_mb = db_size_bytes / (1024 * 1024)
    print(f"Final database file size: {db_size_mb:.2f} MB")
except FileNotFoundError:
    print("Database file not found.")